### Import Libraries and Define Funcs

In [1]:
from omegaconf import OmegaConf, DictConfig
import hydra

from dataclasses import dataclass
from typing import Any

In [2]:
print_yaml = lambda cfg: print(OmegaConf.to_yaml(cfg))

----------

In [4]:
config = OmegaConf.load('configs/config.yaml')
print_yaml(config)

dataset:
  image:
    size: 124
    channels: 3
  classes: 10
feature_extractor:
  _target_: torchvision.models.alexnet
  pretrained: true
defaults:
- config_schema
- classifier: small



### Hydra Basics

#### Defining Configurations

- __Config Outlines__: We can create a defaults.py file to define classes and their attributes in a hierarchical manner. These class structures can then be saved in the hydra ConfigStore in a global manner, such that these can be called else where too.


- __Config Values__: The values for these attributes are set in the configs directory under yaml files.

In [4]:
@dataclass
class ImageConfig:
    size: int
    channels: int

@dataclass
class DatasetConfig:
    image: ImageConfig
    classes: int

@dataclass
class MainConfig:
    dataset : DatasetConfig
    feature_extractor: Any
    classifier: Any


cs = hydra.core.config_store.ConfigStore()
cs.store(name = "config_schema", node = MainConfig)

In [5]:
for name, node in cs.repo.items():
    print(f'- {name}: {node}')

- hydra: {'config.yaml': ConfigNode(name='config.yaml', node={'defaults': [{'output': 'default'}, {'launcher': 'basic'}, {'sweeper': 'basic'}, {'help': 'default'}, {'hydra_help': 'default'}, {'hydra_logging': 'default'}, {'job_logging': 'default'}, {'callbacks': None}, {'env': 'default'}], 'mode': None, 'searchpath': [], 'run': {'dir': '???'}, 'sweep': {'dir': '???', 'subdir': '???'}, 'hydra_logging': '???', 'job_logging': '???', 'sweeper': '???', 'launcher': '???', 'callbacks': {}, 'help': {'app_name': '???', 'header': '???', 'footer': '???', 'template': '???'}, 'hydra_help': {'hydra_help': '???', 'template': '???'}, 'output_subdir': '.hydra', 'overrides': {'hydra': [], 'task': []}, 'job': {'name': '???', 'chdir': None, 'override_dirname': '???', 'id': '???', 'num': '???', 'config_name': '???', 'env_set': {}, 'env_copy': [], 'config': {'override_dirname': {'kv_sep': '=', 'item_sep': ',', 'exclude_keys': []}}}, 'runtime': {'version': '???', 'version_base': '???', 'cwd': '???', 'config_

#### Loading Configurations

##### Using Hydra Main

This is used for standalone python scripts, especially if you want automatic working directory management and CLI overrides. This DOES NOT work in Jupyter Notebooks.

Calling it on top of a function, the following steps occur:
- Changes to working directory to the current one
- Automatically initiallizes Hydra with the config_path
- Loads the config_name yaml file from the config_path
- Applies CLI overrides 

In [7]:
@hydra.main(version_base=None, config_path='configs', config_name='config')
def main(cfg: DictConfig):
    print_yaml(cfg)

    return cfg

if __name__ == "main":
    main()

##### Using Hydra Initialize and Compose

These are used we want to manually control Hydra's initialization and composition.  Also, this is used when Hydra needs to be initalized dynamically (eg. in multi-threading) or if we need multiple config compositions.

- __initialize__ : Sets up the Hydra's configuration system. Now, Hydra knows where to look for the configurations (the directory). No configurations are loaded yet.

- __compose__ : Loads a specific configuration after Hydra is initialized. Stores a config object (DictConfig) in the ConfigStore and returns the config.

In [8]:
from hydra import initialize, compose

# config_path corresponds to the configuration directory where 
# all configs are stored
initialize(version_base = None, config_path = 'configs')

hydra.initialize()

In [9]:
# config_name is the yaml file in the config_path directory
cfg = compose(config_name = 'config')

c:\Users\Admin\anaconda3\envs\hydra_burner\Lib\site-packages\hydra\_internal\defaults_list.py:251: UserWarning: In 'config': Defaults list is missing `_self_`. See https://hydra.cc/docs/1.2/upgrades/1.0_to_1.1/default_composition_order for more information
  warnings.warn(msg, UserWarning)


In [12]:
print_yaml(cfg)

dataset:
  image:
    size: 124
    channels: 3
  classes: 10
feature_extractor:
  _target_: torchvision.models.alexnet
  pretrained: true
classifier:
  dense1:
    _target_: torch.nn.Linear
    in_features: 9216
    out_features: 100
  dense2:
    _target_: torch.nn.Linear
    in_features: ${classifier.dense1.out_features}
    out_features: ${dataset.classes}



--------

### Plugins

Hydra plugins are extendable components that allow Hydra to support new functionality beyond its core features. They are used to customize, extend, or replace Hydra's built-in behavior in a modular way.

Hydra has a plugin system where different types of plugins handle specific tasks, such as:

- Search Path Plugins → Modify where Hydra looks for configs.
- Launcher Plugins → Control how jobs are launched (e.g., SLURM, AWS).
- Sweeper Plugins → Manage hyperparameter tuning (e.g., Optuna, Ax).
- Logging Plugins → Customize logging.
- Output Directory Plugins → Modify how Hydra manages working directories.

#### SearchPathPlugin

In [3]:
from hydra.core.plugins import Plugins
from hydra.plugins.search_path_plugin import SearchPathPlugin
from hydra.core.config_search_path import ConfigSearchPath

In [4]:
class DebugConfigPlugin(SearchPathPlugin):
    def manipulate_search_path(self, search_path: ConfigSearchPath) -> None:

        print(f'Search Paths Before:')
        for path in search_path.get_path():
            print(' ', path)

        search_path.append(
            provider="hydra_path_extension",
            path="file://hydra-tutorial/config/",
        )

        print(f'\nSearch Paths After:')
        for path in search_path.get_path():
            print(' ', path)

In [5]:
Plugins.instance().register(DebugConfigPlugin)

-------

In [6]:
from hydra import initialize, compose

# config_path corresponds to the configuration directory where 
# all configs are stored
initialize(version_base = None, config_path = 'configs')

Search Paths Before:
  provider=hydra, path=pkg://hydra.conf
  provider=main, path=C:\Users\Admin\Documents\GitHub\hydra-tutorial\configs

Search Paths After:
  provider=hydra, path=pkg://hydra.conf
  provider=main, path=C:\Users\Admin\Documents\GitHub\hydra-tutorial\configs
  provider=hydra_path_extension, path=file://hydra-tutorial/config/


hydra.initialize()

In [16]:
hydra.core.global_hydra.GlobalHydra.instance().clear()